In [2]:
import pandas as pd
#data_df = pd.read_csv('clean_newsgroups.csv')
data_df = pd.read_csv('../Output/PubMedArticle(normalize+lemmatize).csv', encoding='utf-8')
data_df.head(10)

,Unnamed: 0,Document,lemmatized,Target Label,Target Name
0,0,Cancer and cure: A critical analysis. Cancer i...,"['cancer', 'cure', 'critical', 'analysis', 'ca...",1,Cancer
1,1,Tumor microenvironment: recent advances in var...,"['tumor', 'microenvironment', 'recent', 'advan...",1,Cancer
2,2,Global Cancer Incidence and Mortality Rates an...,"['global', 'cancer', 'incidence', 'mortality',...",1,Cancer
3,3,Recent Updates on the Relationship between Can...,"['recent', 'update', 'relationship', 'cancer',...",1,Cancer
4,4,What Is Cancer? This essay focuses on themes i...,"['cancer', 'essay', 'focus', 'theme', 'explain...",1,Cancer
5,5,New methods in the diagnosis of cancer and gen...,"['new', 'method', 'diagnosis', 'cancer', 'gene...",1,Cancer
6,6,Cancer-associated fibroblasts in tumor microen...,"['cancerassociated', 'fibroblast', 'tumor', 'm...",1,Cancer
7,7,"Clinical, Prognostic and Therapeutic Significa...","['clinical', 'prognostic', 'therapeutic', 'sig...",1,Cancer
8,8,Cancer prevention: from 1727 to milestones of ...,"['cancer', 'prevention', 'milestone', 'past', ...",1,Cancer
9,9,NaN,['nan'],1,Cancer


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

train_corpus, test_corpus, train_label_nums, test_label_nums, train_label_names, test_label_names =\
                                 train_test_split(np.array(data_df['lemmatized']), np.array(data_df['Target Label']),
                                                       np.array(data_df['Target Name']), test_size=0.33, random_state=42)

train_corpus.shape, test_corpus.shape

((33500,), (16500,))

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# build BOW features on train articles
tv = TfidfVectorizer(use_idf=True, min_df=0.00002, max_df=0.6)
tv_train_features = tv.fit_transform(train_corpus.astype('U'))

In [5]:
# transform test articles into features
tv_test_features = tv.transform(test_corpus.astype('U'))

In [6]:
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

TFIDF model:> Train features shape: (33500, 85199)  Test features shape: (16500, 85199)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
lr = LogisticRegression(penalty='l2', max_iter=1000, C=1, random_state=42)
lr.fit(tv_train_features, train_label_names)
lr_tfidf_cv_scores = cross_val_score(lr, tv_train_features, train_label_names, cv=5)
lr_tfidf_cv_mean_score = np.mean(lr_tfidf_cv_scores)
print('CV Accuracy (5-fold):', lr_tfidf_cv_scores)
print('Mean CV Accuracy:', lr_tfidf_cv_mean_score)
lr_tfidf_test_score = lr.score(tv_test_features, test_label_names)
print('Test Accuracy:', lr_tfidf_test_score)

CV Accuracy (5-fold): [0.82701493 0.82208955 0.81686567 0.82402985 0.83268657]
Mean CV Accuracy: 0.8245373134328359
Test Accuracy: 0.8306666666666667


In [8]:
from sklearn.svm import LinearSVC

svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(tv_train_features, train_label_names)
svm_tfidf_cv_scores = cross_val_score(svm, tv_train_features, train_label_names, cv=5)
svm_tfidf_cv_mean_score = np.mean(svm_tfidf_cv_scores)
print('CV Accuracy (5-fold):', svm_tfidf_cv_scores)
print('Mean CV Accuracy:', svm_tfidf_cv_mean_score)
svm_tfidf_test_score = svm.score(tv_test_features, test_label_names)
print('Test Accuracy:', svm_tfidf_test_score)

CV Accuracy (5-fold): [0.82641791 0.82313433 0.81656716 0.82402985 0.83253731]
Mean CV Accuracy: 0.8245373134328358
Test Accuracy: 0.830969696969697


In [9]:
pd.DataFrame([['Logistic Regression', 
               lr_tfidf_cv_mean_score, lr_tfidf_test_score],
              ['Linear SVM', 
               svm_tfidf_cv_mean_score, svm_tfidf_test_score]],
             columns=['Model',  'CV Score (TF-IDF)', 'Test Score (TF-IDF)'],
             )

,Model,CV Score (TF-IDF),Test Score (TF-IDF)
0,Logistic Regression,0.824537,0.830667
1,Linear SVM,0.824537,0.830970
